In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin
import datetime
import pymysql

#### 네이버 환율 일별시세

- 미국(USD), 유럽연합(EUR), 일본(JPY), 중국(CNY) 통화의 매매기준율 추출

In [6]:
def exchange_day_sise_crawler(country, base_url):

    temp_dict = {}

    usaList = []
    eurList = []
    jpyList = []
    cnyList = []

    for country_name, country_code in country.items():
        dateList = []
        for i in range(1,4):
            params = {
                'marketindexCd': country_code,
            }

            resp = requests.get(base_url, params=params)
            soup = BeautifulSoup(resp.text,'lxml')

            _iframe = soup.find_all('iframe')
            day_sise = urljoin(base_url,_iframe[1].get('src'))
            day_sise = day_sise+'&page='+str(i)
#             print(day_sise)

            resp2 = requests.get(day_sise)
            soup2 = BeautifulSoup(resp2.text,'lxml')

            day_body = soup2.select('tbody tr')
            
            # 각 나라별 환율정보 저장
            for body in day_body:
                dateList.append(body.find('td', class_='date').text.replace('.','-'))

                if country_name == 'USA':
                    usaList.append(body.find('td', class_='num').text)

                elif country_name == 'EUR':
                    eurList.append(body.find('td', class_='num').text)

                elif country_name == 'JPY':
                    jpyList.append(body.find('td', class_='num').text)

                elif country_name == 'CNY':
                    cnyList.append(body.find('td', class_='num').text)
    
    df1 = pd.DataFrame({'날짜' : dateList,
                        'USA' : usaList,
                        'EUR' : eurList,
                        'JPY' : jpyList,
                        'CNY' : cnyList})
    
    return df1

#### 수집해야할 모든날짜 DataFrame 만들기 (공휴일 포함)

In [7]:
def all_date_range_df(date_range):
    
    df2 = pd.DataFrame({'날짜':date_range})
    
    return df2

#### 두개의 DataFrame 병합

In [8]:
def df_outer_join_merge(df1, df2):
    
    # 두 df을 '날짜'키 값을 기준으로 outer_join 병합
    df_OUTER_JOIN = pd.merge(df1, df2, left_on='날짜', right_on='날짜', how='outer', sort=True)
    
    # 두 값중 결측값을 앞 방향으로 채우기 (공휴일에 없는 환율정보는 전날을 기준으로 결측값 채우기)
    # https://rfriend.tistory.com/262
    df_finally = df_OUTER_JOIN.fillna(method='ffill')
    
    return df_finally

In [9]:
def save_df_to_csv(df_finally):
    # 위에 4월달꺼 제외 후 저장
    
    df_finally[4:].to_csv('naver_exchange_crawling.csv', index=False)

In [10]:
country = {'USA':'FX_USDKRW', 'EUR':'FX_EURKRW', 'JPY':'FX_JPYKRW', 'CNY':'FX_CNYKRW'}
base_url = 'https://finance.naver.com/marketindex/exchangeDetail.nhn'

date_range = (pd.date_range(start='20200501', end=datetime.date.today())).strftime("%Y-%m-%d").tolist()

ExchangeDf = exchange_day_sise_crawler(country, base_url)
DateDf = all_date_range_df(date_range)
FinallyDf = df_outer_join_merge(ExchangeDf, DateDf)
# save_df_to_csv(FinallyDf)

###  https://godoftyping.wordpress.com/2017/05/27/python-mysql/ 
pymysql 참고할 블로그

# 위에서 수집한 환율 데이터 mariaDB에 저장하기

In [104]:
import pymysql

#####  DB  생성 (CREATE 사용)

In [106]:
def make_DB(password, DB_name):
    conn = pymysql.connect(host='localhost', 
                           user = 'root', 
                           password=password, 
                           charset='utf8',
                           cursorclass=pymysql.cursors.DictCursor)
    try:
        with conn.cursor() as cursor:
            print('creating DataBase ...')
            sql = 'CREATE DATABASE '+ DB_name
            cursor.execute(sql)
        conn.commit()
    finally:
        conn.close()

#####  table  생성 (CREATE 사용)

In [103]:
# 'multi_crawler' DB에 'naver_exchange' 테이블 생성

def make_table(password, DB_name, table_name):
    conn = pymysql.connect(host='localhost', 
                           user = 'root', 
                           password=password, 
                           db=DB_name, 
                           charset='utf8',
                           cursorclass=pymysql.cursors.DictCursor)
    try:
        with conn.cursor() as cursor:
            print('creating table ...')
            # 테이블이 생성되지 않은 경우만 테이블 생성
            sql = 'CREATE TABLE IF NOT EXISTS '+table_name+'''(날짜 CHAR(10) PRIMARY KEY,
                                            USA DECIMAL(10) NOT NULL, 
                                            EUR DECIMAL(10) NOT NULL,
                                            JPY DECIMAL(10) NOT NULL,
                                            CNY DECIMAL(10) NOT NULL)'''
            cursor.execute(sql)
        conn.commit()
    finally:
        conn.close()    

creating table ...


#####  table  전체 삭제 (Drop 사용)

In [ ]:
# 테이블 내용 전체 삭제 

def drop_table(table_name):
    print('delete table ...')
    sql  = "DROP TABLE IF EXISTS "+ table_name
    curs.execute(sql)

In [ ]:
# insert문을 사용해서 데이터 입력
def input_date():
    print('inserting data ...')
    
    # FOR LOOP로 바꿔서 수집한 데이터 테이블에  INSERT하기
    sql = "INSERT INTO naver_exchange VALUES (%s, %s, %s, %s, %s)"
    for index, row in FinallyDf[4:].iterrows():
#         print(row['날짜'], row['USA'].replace(',',''),row['EUR'].replace(',',''),row['JPY'].replace(',',''),row['CNY'].replace(',',''))
        val = (row['날짜'], row['USA'].replace(',',''), row['EUR'].replace(',',''), row['JPY'].replace(',',''), row['CNY'].replace(',',''))

        try:
            curs.execute(sql, val)

        except:
            print(curs._last_executed)
    
    conn.commit()
    print(curs.rowcount, "record inserted.")

### main.py

In [ ]:
password = '12345'
DB_name = 'multi_crawler'
table_name = 'naver'

# DB 생성
make_DB(password, DB_name)
# table 생성
make_table(password, DB_name, table_name)
# table 삭제
drop_table(table_name)

In [ ]:
# 데이터 가져오기
with conn.cursor() as cursor:
    sql = "SELECT * FROM table"
    cursor.execute(sql)
    data = cursor.fetchall()
    print(data)

In [100]:
# MairaDB  connection 연결
conn = pymysql.connect(host='localhost', 
                       user = 'root', 
                       password='12345', 
                       db='multi_crawler2', 
                       charset='utf8',
                       cursorclass=pymysql.cursors.DictCursor)

# connection 으로부터 Dictionary Cursor 생성
# curs = conn.cursor(pymysql.cursors.DictCursor)
curs = conn.cursor()

# 'multi_crawler' DB에 'naver_exchange' 테이블 생성
try:
    print('creating table ...')
    # 테이블이 생성되지 않은 경우만 테이블 생성
    sql = '''CREATE TABLE IF NOT EXISTS naver_exchange(날짜 CHAR(10), 
                                        USA DECIMAL(10) AUTO_INCREMENT PRIMARY KEY, 
                                        EUR DECIMAL(10) NOT NULL, 
                                        JPY DECIMAL(10) NOT NULL, 
                                        CNY DECIMAL(10) NOT NULL)'''
    curs.execute(sql)

    # 테이블 생성 확인
    curs.execute("SHOW TABLES")

    # insert문을 사용해서 데이터 입력
    print('inserting data ...')
    
    # FOR LOOP로 바꿔서 수집한 데이터 테이블에  INSERT하기
    sql = "INSERT INTO naver_exchange VALUES (%s, %s, %s, %s, %s)"
    for index, row in FinallyDf[4:].iterrows():
        print(row['날짜'], row['USA'].replace(',',''),row['EUR'].replace(',',''),row['JPY'].replace(',',''),row['CNY'].replace(',',''))
        val = (row['날짜'], row['USA'].replace(',',''), row['EUR'].replace(',',''), row['JPY'].replace(',',''), row['CNY'].replace(',',''))

        try:
            curs.execute(sql, val)

        except:
            print(curs._last_executed)
    
    conn.commit()
    print(curs.rowcount, "record inserted.")
    
except:
    print('error.....')


# 테이블 내용 전체 삭제 
# print('delete table ...')
# sql  = ("DROP TABLE IF EXISTS naver_exchange")
# curs.execute(sql)
# # 테이블 생성 확인
# curs.execute("SHOW TABLES")

creating table ...
inserting data ...
2020-05-01 1219.00 1344.56 1144.55 172.06
2020-05-02 1219.00 1344.56 1144.55 172.06
['DataError', 'DatabaseError', 'Error', 'IntegrityError', 'InterfaceError', 'InternalError', 'NotSupportedError', 'OperationalError', 'ProgrammingError', 'Warning', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_check_executed', '_clear_result', '_conv_row', '_defer_warnings', '_do_execute_many', '_do_get_result', '_ensure_bytes', '_escape_args', '_executed', '_get_db', '_last_executed', '_nextset', '_query', '_result', '_rows', '_show_warnings', '_warnings_handled', 'arraysize', 'callproc', 'close', 'connection', 'description', 'execut

C:\Users\student\.conda\envs\multi_python\lib\site-packages\pymysql\cursors.py:170: Warning: (1265, "Data truncated for column 'USA' at row 1")
  result = self._query(query)


In [ ]:
# ORM ->  추가적인 공부가 필요(sql rpa)

class table_name(sa.Model):
    data_id = sa.Integer(primary = True, auto_increment= True)
    content = sa.Text()
    source_id = sa.ForeignKey()

In [ ]:
# 명령어 작성 :  python  sa migrate  ()